In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K

In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
def identity_block(x, filter):

    x_skip = x

    x =  keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x =  keras.layers.BatchNormalization(axis=3)(x)
    x =  keras.layers.Activation('relu')(x)

    x =  keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x =  keras.layers.BatchNormalization(axis=3)(x)

    x =  keras.layers.Add()([x, x_skip])     
    x =  keras.layers.Activation('relu')(x)
    
    return x

In [ ]:
def convolutional_block(x, filter):

    x_skip = x

    x =  keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x =  keras.layers.BatchNormalization(axis=3)(x)
    x =  keras.layers.Activation('relu')(x)

    x =  keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x =  keras.layers.BatchNormalization(axis=3)(x)

    x_skip =  keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)

    x =  keras.layers.Add()([x, x_skip])     
    x =  keras.layers.Activation('relu')(x)
    
    return x

In [ ]:
def ResNet34(shape = (96, 96, 3), classes = 1):

    x_input = keras.layers.Input(shape)
    x = keras.layers.ZeroPadding2D((3, 3))(x_input)

    x = keras.layers.Conv2D(32, kernel_size=7, strides=2, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)

    block_layers = [3, 4, 6, 3]
    filter_size = 32

    for i in range(4):
        if i == 0:
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)

    x = keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(512, activation = 'relu')(x)
    x = keras.layers.Dense(classes, activation = 'sigmoid')(x)
    model = keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    model.compile(loss=compile_loss, optimizer=lr, metrics=['accuracy', get_f1])
    return model

In [ ]:
data_dir_train = 'Dataset/train/'
data_dir_test = 'Dataset/test/'

img_size = 256
input_shape = (img_size, img_size, 1)
target_size =(img_size,img_size)
batch_size = 32
num_classes = 1

test_num = 50

compile_optimizer = "adam"
compile_loss = "binary_crossentropy"
learning_rate= 0.001
lr = keras.optimizers.Adam(learning_rate=learning_rate)

csv_path = f'Results/Dataset_ResNet34_{batch_size}_{img_size}_Test_{test_num}_Learning_rate{learning_rate}.csv'
hist_path = f'Results/Dataset_ResNet34_{batch_size}_{img_size}_Test_{0}_Learning_rate{learning_rate}_history.csv'
save_model_path = f"Models/Dataset_ResNet34_{batch_size}_{img_size}_Test_{test_num}_Learning_rate{learning_rate}.h5"

In [ ]:
training_data = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 25,
        width_shift_range = 0.25,
        height_shift_range = 0.25,
        shear_range = 0.5,
        zoom_range = 0.25,
        brightness_range= [0.6,0.9],
        vertical_flip = True,
        fill_mode = 'nearest',
        validation_split = 0.2,
        )

test_data = ImageDataGenerator(
        rescale = 1./255,
        )

In [ ]:
train_ds = training_data.flow_from_directory(
    directory=data_dir_train,
    shuffle = True,
    seed = 42,
    color_mode="grayscale",
    class_mode = 'binary',
    target_size=target_size,
    batch_size=batch_size,
    subset="training",
   )

validation_ds = training_data.flow_from_directory(
    directory=data_dir_train,
    shuffle = True,
    seed = 42,
    class_mode = 'binary',
    color_mode="grayscale",
    target_size=target_size,
    batch_size=batch_size,
    subset="validation",
    )

test_ds = test_data.flow_from_directory(
    directory=data_dir_test,
    target_size=target_size,
    color_mode="grayscale",
    shuffle = False,
    )

In [ ]:
model = ResNet34(input_shape)
#model.load_weights('Models/DatasetV2_ResNet34_32_200_Test_1.h5')
#model.load_weights(save_model_path)
model.summary()

In [ ]:
checkpoint = ModelCheckpoint(save_model_path, monitor='val_accuracy',
                             save_best_only=True, save_weights_only=True, mode='auto')

early = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, mode="min")

history = model.fit(train_ds, epochs=50,  callbacks=[checkpoint,early], validation_data = validation_ds)

In [ ]:
model.load_weights(save_model_path)
#history = pd.read_csv(hist_path)
#df = pd.DataFrame(history)
#df.plot(figsize=(10,8))
#df.to_csv(hist_path)
#train_data = model.evaluate(train_ds)
#val_data = model.evaluate(validation_ds)
#log = f'Dataset_2_ResNet34_{batch_size}_{img_size}_Test_{test_num} : '+ str(val_data)
#logging.info(log)

In [ ]:
history = pd.read_csv(hist_path)
df = pd.DataFrame(history[['get_f1', 'val_get_f1']])
df.plot(figsize=(10,8))

In [ ]:
Y_pred = model.predict(validation_ds, 1496 // batch_size+1)
cf_matrix = confusion_matrix(validation_ds.classes, np.round(Y_pred))

In [ ]:
cf_matrix = confusion_matrix(validation_ds.classes, np.round(Y_pred))

In [ ]:
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
pred = model.predict(test_ds, steps = len(test_ds), verbose=1)
cl = np.round(pred)
classes_prediction = []
for prediction in cl[:,0]:
    if (prediction == 0.0):
        classes_prediction.append('Cracked')
    elif(prediction == 1.0):
        classes_prediction.append('Uncracked')

filenames=test_ds.filenames
results=pd.DataFrame({"file":filenames,"prediction":pred[:,0], "class":classes_prediction})

In [ ]:
results.to_csv(csv_path)
#results.style

In [ ]:
import cv2 
rez = pd.read_csv(csv_path)
plt.figure(figsize=(40,28))
for i in range(56):
    img_path = str(rez['file'][i])
    image_path = data_dir_test + 'test/' + img_path[5:]
    #print(image_path)
    class_pred = str(rez['class'][i])
    img = cv2.imread(image_path)
    plt.subplot(7,8, i+1)
    plt.imshow(img)
    plt.title(class_pred)

In [ ]:
print(classes_prediction.count("Cracked"))